In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
tweets_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [3]:
tweets_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Drop the columns without information

In [4]:
tweets_train = tweets_train.drop(["id","location"],axis=1)

In [5]:
tweets_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   keyword  7552 non-null   object
 1   text     7613 non-null   object
 2   target   7613 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 178.6+ KB


In [6]:
tweets_train["text"] = tweets_train["text"].astype("string")

In [7]:
tweets_train["text"] = tweets_train["text"].apply(lambda x: x.lower())

In [8]:
import re
import string
import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
from nltk.stem.snowball import SnowballStemmer

In [9]:

translator = str.maketrans(" "," ",string.punctuation)
translator_number = str.maketrans(" "," ","0123456789")

In [10]:
def remove_http(string):
    return re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", string)

def remove_accents(string):
    return unidecode.unidecode(string)

def remove_stopwords(string):
    tokens = word_tokenize(string)
    filtred = [word for word in tokens if not word in get_stop_words("en")]
    return " ".join(filtred)


In [11]:
tweets_train["text"] = tweets_train["text"].apply(remove_accents)
tweets_train["text"] = tweets_train["text"].apply(remove_http)
tweets_train["text"] = tweets_train["text"].apply(lambda x: x.translate(translator))
tweets_train["text"] = tweets_train["text"].apply(lambda x: x.translate(translator_number))
tweets_train["text"] = tweets_train["text"].apply(remove_stopwords)


In [12]:
tweets_train["text"]

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       people receive wildfires evacuation orders cal...
4       just got sent photo ruby alaska smoke wildfire...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    thetawniest control wild fires california even...
7610                            m utc km s volcano hawaii
7611    police investigating e bike collided car littl...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [13]:
lemm = WordNetLemmatizer()
snow_stemmer = SnowballStemmer(language="english")
def Lemmatizer(string):
    tokens = word_tokenize(string)
    filtred = [lemm.lemmatize(word) for word in tokens]
    filtred_f = [snow_stemmer.stem(word) for word in filtred]
    return " ".join(filtred_f)

In [14]:
tweets_train["text"] = tweets_train["text"].apply(Lemmatizer)


In [15]:
from sklearn.feature_extraction.text import CountVectorizer 

In [16]:
Count_V = CountVectorizer(input="content")
X = Count_V.fit_transform(np.array(tweets_train["text"]))

In [17]:
tweets_train["text"].head(50)

0              deed reason earthquak may allah forgiv u
1                  forest fire near la rong sask canada
2     resid ask shelter place notifi offic evacu she...
3           peopl receiv wildfir evacu order california
4     just got sent photo rubi alaska smoke wildfir ...
5     rockyfir updat california hwi close direct due...
6     flood disast heavi rain caus flash flood stree...
7                          m top hill can see fire wood
8          s emerg evacu happen now build across street
9                            m afraid tornado come area
10                        three peopl die heat wave far
11    haha south tampa get flood hah wait second liv...
12    rain flood florida tampabay tampa day ve lost ...
13                        flood bago myanmar arriv bago
14               damag school bus multi car crash break
15                                                s man
16                                           love fruit
17                                          summ

In [18]:
type(X)

scipy.sparse.csr.csr_matrix

In [19]:
df = pd.DataFrame(X.toarray(),columns=Count_V.get_feature_names())